This module:

    ** 1. Merge the pumped signals with time stamps to get vertical format for voting
    ** 2. Append the output whethere the coin was pumped or not (binary 0)

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
import pickle
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

## Import the pumped data

### Step 1. Create the timestamp for the first hours 

In [2]:
p = pd.read_pickle("../pump_data_with_social.pkl")
p = p[['channelLink','channelTitle','currency','exchange','signalTime','pumptime_edited','timestamp']]
p.head(2)

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032


Ther are .... unique points in time...

In [3]:
#Number of unique time points
len(pd.unique(p['timestamp']))

422

In [4]:
#Number of unique coins points
len(pd.unique(p['currency']))

154

In [5]:
3600*24

86400

In [6]:
1562346032 - (1562346032 % (86400))

1562284800

..Strange beheavior, must write each function one by one...

In [7]:
def getDaystamp(dfo: pd.DataFrame, num_days: int=1, var:str='timestamp'):
    '''
    Append a column that contains the timestamp at 12:00 AM (0:00) of the
    original var variable.
    :param df: dataframe in question 
    :param num_date: number of dates to go back to
    :param var str: name of the variable that has original timestamp
    '''
    df = dfo.copy()
    df[var+'_int'] = df[var].apply(int)
    for d in range(num_days+1):
        df[var+'_d'+str(d)] = df[var+'_int'].apply(lambda t : t - (t % (3600*24)) - d*(3600*24))
    df = df.drop(columns=var+'_int')
    return df

In [8]:
num_days = 31
p = getDaystamp(p,num_days)
p.head()

,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_d0,timestamp_d1,timestamp_d2,timestamp_d3,timestamp_d4,timestamp_d5,timestamp_d6,timestamp_d7,timestamp_d8,timestamp_d9,timestamp_d10,timestamp_d11,timestamp_d12,timestamp_d13,timestamp_d14,timestamp_d15,timestamp_d16,timestamp_d17,timestamp_d18,timestamp_d19,timestamp_d20,timestamp_d21,timestamp_d22,timestamp_d23,timestamp_d24,timestamp_d25,timestamp_d26,timestamp_d27,timestamp_d28,timestamp_d29,timestamp_d30,timestamp_d31
0,https://t.me/crypto_pump_island,Crypto Pump Island,ADX,Binance,2019-07-26T17:59:59Z,2019-07-26 17:59:59+00:00,1564163999,1564099200,1564012800,1563926400,1563840000,1563753600,1563667200,1563580800,1563494400,1563408000,1563321600,1563235200,1563148800,1563062400,1562976000,1562889600,1562803200,1562716800,1562630400,1562544000,1562457600,1562371200,1562284800,1562198400,1562112000,1562025600,1561939200,1561852800,1561766400,1561680000,1561593600,1561507200,1561420800
1,https://t.me/SignalsKingdom,Signal Kingdom,ADX,Binance,2019-07-05T17:00:32,2019-07-05 17:00:32+00:00,1562346032,1562284800,1562198400,1562112000,1562025600,1561939200,1561852800,1561766400,1561680000,1561593600,1561507200,1561420800,1561334400,1561248000,1561161600,1561075200,1560988800,1560902400,1560816000,1560729600,1560643200,1560556800,1560470400,1560384000,1560297600,1560211200,1560124800,1560038400,1559952000,1559865600,1559779200,1559692800,1559606400
2,https://t.me/MaximumPump,Maximum Pump,ADX,Binance,2019-04-25T19:15:21,2019-04-25 19:15:21+00:00,1556219721,1556150400,1556064000,1555977600,1555891200,1555804800,1555718400,1555632000,1555545600,1555459200,1555372800,1555286400,1555200000,1555113600,1555027200,1554940800,1554854400,1554768000,1554681600,1554595200,1554508800,1554422400,1554336000,1554249600,1554163200,1554076800,1553990400,1553904000,1553817600,1553731200,1553644800,1553558400,1553472000
3,https://t.me/bitcoinpumpgroup,Bitcoin Pump Group,WAN,Binance,2019-08-30T16:41:30Z,2019-08-30 16:41:30+00:00,1567183290,1567123200,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800
4,https://t.me/bitcoinprofitcoach,Bitcoin Profit Coach,QSP,Binance,2019-08-29T19:46:43Z,2019-08-29 19:46:43+00:00,1567108003,1567036800,1566950400,1566864000,1566777600,1566691200,1566604800,1566518400,1566432000,1566345600,1566259200,1566172800,1566086400,1566000000,1565913600,1565827200,1565740800,1565654400,1565568000,1565481600,1565395200,1565308800,1565222400,1565136000,1565049600,1564963200,1564876800,1564790400,1564704000,1564617600,1564531200,1564444800,1564358400


### Step 2: Loop throug the processed social data file and merge based on timestemp
... We do not need to join by the coinID. Just the timestamps are good enough. 
Since we want to retrieve the values for all availabel timestamps

In [9]:
# dir_prefix = "../../Social_Data/Daily/"
# social_error = []
# x_final = pd.DataFrame()
# t_temp = pd.DataFrame()
# for filename in os.listdir(dir_prefix):
#     if 'pkl' in filename:
#         _,coin_name, coin_id = filename.split('_')
#         try: 
#             print("Processing file...",filename)
#             l = p[p.currency==coin_name]
#             for d in range(num_days+1):
#                 if 'timestamp_d'+str(d) not in l.columns: 
#                     print('Info. on day_{} not in the left table!'.format(d))
#                     continue
#                 x = pd.read_pickle(dir_prefix+filename)
#                 x.columns = [x+'_d'+str(d) if x != 'time' else 'time' for x in x.columns]
#                 x['coin_name'] = coin_name
#                 x['coin_id'] = coin_id
#                 l = pd.merge(left=l,left_on=['timestamp_d'+str(d),'currency']
#                              ,right=x, right_on=["time",'coin_name']
#                              ,how='inner')
#                 l = l.drop(columns=['coin_name','coin_id','time'])
#             #Save the file in the processed_directory
#             l.to_pickle(dir_prefix+'/Processed_Data/d'+str(num_days)+filename)
#             #Append all the data on top of each other for processing; 
#             x_final = x_final.append(l)
#         except: 
#             soc/ial_error.append(filename)
#             continue

## DAILY

In [10]:
dir_prefix = "../../Social_Data/Daily/Processed Data/Time Series/"
social_error = []
x_final = pd.DataFrame()
t_temp = pd.DataFrame()
for filename in os.listdir(dir_prefix):
    if 'pkl' in filename:
#         print(filename)
        _, coin_name, coin_id, _ = filename.split('_')
        try: 
            print("Processing file...",filename)
            l = p
            for d in range(num_days+1):
                if 'timestamp_d'+str(d) not in l.columns: 
                    print('Info. on day_{} not in the left table!'.format(d))
                    continue
                x = pd.read_pickle(dir_prefix+filename)
                x.columns = [x+'_d'+str(d) if x != 'time' else 'time' for x in x.columns]
                l = pd.merge(left=l,left_on=['timestamp_d'+str(d)]
                             ,right=x, right_on=["time"]
                             ,how='inner')
                l = l.drop(columns=['time'])
            #Save the file in the processed_directory
            l['coin_name'] = coin_name
            l['coin_id'] = coin_id.replace('.pkl','')
            #Append all the data on top of each other for processing; 
            x_final = x_final.append(l)
        except: 
            social_error.append(filename)
            continue

Processing file... Daily_POA_793845_Time.pkl
Processing file... Daily_BRX_299415_Time.pkl
Processing file... Daily_VIBE_336538_Time.pkl
Processing file... Daily_MATIC_930246_Time.pkl
Processing file... Daily_FTM_928158_Time.pkl
Processing file... Daily_PTOY_129035_Time.pkl
Processing file... Daily_NKT_5334_Time.pkl
Processing file... Daily_PAX_925939_Time.pkl
Processing file... Daily_FUN_178978_Time.pkl
Processing file... Daily_WPR_355242_Time.pkl
Processing file... Daily_NULS_377792_Time.pkl
Processing file... Daily_POLY_744820_Time.pkl
Processing file... Daily_VIB_198710_Time.pkl
Processing file... Daily_HQX_535054_Time.pkl
Processing file... Daily_QKC_899553_Time.pkl
Processing file... Daily_NAS_619555_Time.pkl
Processing file... Daily_BNT_22327_Time.pkl
Processing file... Daily_HC_926044_Time.pkl
Processing file... Daily_SIG_867303_Time.pkl
Processing file... Daily_GAM_6102_Time.pkl
Processing file... Daily_MDA_341279_Time.pkl
Processing file... Daily_LINK_309621_Time.pkl
Processin

Processing file... Daily_MNZ_341059_Time.pkl
Processing file... Daily_ARDR_30173_Time.pkl
Processing file... Daily_STEEM_20333_Time.pkl
Processing file... Daily_NPXS_731516_Time.pkl
Processing file... Daily_AGI_710156_Time.pkl
Processing file... Daily_GET_597181_Time.pkl
Processing file... Daily_CVC_139467_Time.pkl


In [11]:
print(social_error)
print(l.shape)

[]
(426, 1225)


#We might have to check whether there is a non-zero data somewhere....

In [12]:
x_final = x_final.reset_index(drop=True)
x_final = x_final.sort_values('timestamp')
x_final = x_final.drop_duplicates()

In [13]:
x_final.filter(regex="twitter_fav")

,twitter_favourites_d0,twitter_favourites_d1,twitter_favourites_d2,twitter_favourites_d3,twitter_favourites_d4,twitter_favourites_d5,twitter_favourites_d6,twitter_favourites_d7,twitter_favourites_d8,twitter_favourites_d9,twitter_favourites_d10,twitter_favourites_d11,twitter_favourites_d12,twitter_favourites_d13,twitter_favourites_d14,twitter_favourites_d15,twitter_favourites_d16,twitter_favourites_d17,twitter_favourites_d18,twitter_favourites_d19,twitter_favourites_d20,twitter_favourites_d21,twitter_favourites_d22,twitter_favourites_d23,twitter_favourites_d24,twitter_favourites_d25,twitter_favourites_d26,twitter_favourites_d27,twitter_favourites_d28,twitter_favourites_d29,twitter_favourites_d30,twitter_favourites_d31
40469,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65
29393,465,465,465,465,465,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,466,465,465,0,0,0,0,0,0,0
28967,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
28541,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,7,7,6
28115,152,149,149,147,146,145,145,145,145,145,145,144,142,142,142,142,140,140,139,139,139,139,139,139,139,134,133,131,130,129,119,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50274,1907,1907,1907,1907,1906,1906,1908,1908,1908,1908,1907,1907,1907,1907,1907,1908,1908,1908,1904,1904,1904,1905,1905,1905,1906,1906,1905,1905,1905,1905,1905,1905
69870,807,807,807,807,807,807,807,807,808,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807,807
18750,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


*** Now we have the time series data up to 1 month in the past!
*** So now we can start preparing the data and sort the categories together?

In [14]:
print(x_final.shape)
x_final[-5:]

(80940, 1225)


,channelLink,channelTitle,currency,exchange,signalTime,pumptime_edited,timestamp,timestamp_d0,timestamp_d1,timestamp_d2,timestamp_d3,timestamp_d4,timestamp_d5,timestamp_d6,timestamp_d7,timestamp_d8,timestamp_d9,timestamp_d10,timestamp_d11,timestamp_d12,timestamp_d13,timestamp_d14,timestamp_d15,timestamp_d16,timestamp_d17,timestamp_d18,timestamp_d19,timestamp_d20,timestamp_d21,timestamp_d22,timestamp_d23,timestamp_d24,timestamp_d25,timestamp_d26,timestamp_d27,timestamp_d28,timestamp_d29,timestamp_d30,timestamp_d31,comments_d0,posts_d0,followers_d0,points_d0,overview_page_views_d0,analysis_page_views_d0,markets_page_views_d0,charts_page_views_d0,trades_page_views_d0,forum_page_views_d0,influence_page_views_d0,total_page_views_d0,fb_likes_d0,fb_talking_about_d0,twitter_followers_d0,twitter_following_d0,twitter_lists_d0,twitter_favourites_d0,twitter_statuses_d0,reddit_subscribers_d0,reddit_active_users_d0,reddit_posts_per_hour_d0,reddit_posts_per_day_d0,reddit_comments_per_hour_d0,reddit_comments_per_day_d0,code_repo_stars_d0,code_repo_forks_d0,code_repo_subscribers_d0,code_repo_open_pull_issues_d0,code_repo_closed_pull_issues_d0,code_repo_open_issues_d0,code_repo_closed_issues_d0,code_repo_contributors_d0,date_d0,date_utc_d0,coin_name_d0,coin_id_d0,comments_d1,posts_d1,followers_d1,points_d1,overview_page_views_d1,analysis_page_views_d1,markets_page_views_d1,charts_page_views_d1,trades_page_views_d1,forum_page_views_d1,influence_page_views_d1,total_page_views_d1,fb_likes_d1,fb_talking_about_d1,twitter_followers_d1,twitter_following_d1,twitter_lists_d1,twitter_favourites_d1,twitter_statuses_d1,reddit_subscribers_d1,reddit_active_users_d1,reddit_posts_per_hour_d1,reddit_posts_per_day_d1,reddit_comments_per_hour_d1,reddit_comments_per_day_d1,code_repo_stars_d1,code_repo_forks_d1,code_repo_subscribers_d1,code_repo_open_pull_issues_d1,code_repo_closed_pull_issues_d1,code_repo_open_issues_d1,code_repo_closed_issues_d1,code_repo_contributors_d1,date_d1,date_utc_d1,coin_name_d1,coin_id_d1,comments_d2,posts_d2,followers_d2,points_d2,overview_page_views_d2,analysis_page_views_d2,markets_page_views_d2,charts_page_views_d2,trades_page_views_d2,forum_page_views_d2,influence_page_views_d2,total_page_views_d2,fb_likes_d2,fb_talking_about_d2,twitter_followers_d2,twitter_following_d2,twitter_lists_d2,twitter_favourites_d2,twitter_statuses_d2,reddit_subscribers_d2,reddit_active_users_d2,reddit_posts_per_hour_d2,reddit_posts_per_day_d2,reddit_comments_per_hour_d2,reddit_comments_per_day_d2,code_repo_stars_d2,code_repo_forks_d2,code_repo_subscribers_d2,code_repo_open_pull_issues_d2,code_repo_closed_pull_issues_d2,code_repo_open_issues_d2,code_repo_closed_issues_d2,code_repo_contributors_d2,date_d2,date_utc_d2,coin_name_d2,coin_id_d2,comments_d3,posts_d3,followers_d3,points_d3,overview_page_views_d3,analysis_page_views_d3,markets_page_views_d3,charts_page_views_d3,trades_page_views_d3,forum_page_views_d3,influence_page_views_d3,total_page_views_d3,fb_likes_d3,fb_talking_about_d3,twitter_followers_d3,twitter_following_d3,twitter_lists_d3,twitter_favourites_d3,twitter_statuses_d3,reddit_subscribers_d3,reddit_active_users_d3,reddit_posts_per_hour_d3,reddit_posts_per_day_d3,reddit_comments_per_hour_d3,reddit_comments_per_day_d3,code_repo_stars_d3,code_repo_forks_d3,code_repo_subscribers_d3,code_repo_open_pull_issues_d3,code_repo_closed_pull_issues_d3,code_repo_open_issues_d3,code_repo_closed_issues_d3,code_repo_contributors_d3,date_d3,date_utc_d3,coin_name_d3,coin_id_d3,comments_d4,posts_d4,followers_d4,points_d4,overview_page_views_d4,analysis_page_views_d4,markets_page_views_d4,charts_page_views_d4,trades_page_views_d4,forum_page_views_d4,influence_page_views_d4,total_page_views_d4,fb_likes_d4,fb_talking_about_d4,twitter_followers_d4,twitter_following_d4,twitter_lists_d4,twitter_favourites_d4,twitter_statuses_d4,reddit_subscribers_d4,reddit_active_users_d4,reddit_posts_per_hour_d4,reddit_posts_per_day_d4,reddit_comments_per_hour_d4,reddit_comments_per_

Addiitional steps to process data:
* Group the data by types of social media

In [15]:
code = ['code_repo_stars',
'code_repo_forks' ,
'code_repo_subscribers',
'code_repo_contributors',
'code_repo_open_pull_issues',
'code_repo_closed_pull_issues',
'code_repo_open_issues',
'code_repo_closed_issues']

twitter = ['twitter_followers',
'twitter_following',
'twitter_lists',
'twitter_favourites',
'twitter_statuses']

reddit = ['reddit_subscribers',
'reddit_active_users',
'reddit_posts_per_hour',
'reddit_posts_per_day',
'reddit_comments_per_hour',
'reddit_comments_per_day'
]

fb = ['fb_likes','fb_talking_about']

market = [
'comments',
'posts',
'followers' ,
'points',
'overview_page_views',
'analysis_page_views',
'markets_page_views',
'charts_page_views',
'trades_page_views',
'forum_page_views',
'influence_page_views',
'total_page_views'
]


In [16]:
d = 32
col_dict = dict()
for f in ['code','twitter','reddit','fb','market']:
    dt = dict()
    for i in eval(f):
        dt[i] = [i+"_d"+str(y) for y in range(d)]
    col_dict[f] = dt

In [17]:
core = ['channelLink','channelTitle','currency','exchange',
        'timestamp','signalTime','pumptime_edited','coin_name', 'coin_id']

In [18]:
col_dict['core'] = core

In [19]:
col_dict[list(col_dict.keys())[0]].keys()

dict_keys(['code_repo_stars', 'code_repo_forks', 'code_repo_subscribers', 'code_repo_contributors', 'code_repo_open_pull_issues', 'code_repo_closed_pull_issues', 'code_repo_open_issues', 'code_repo_closed_issues'])

In [20]:
code_d = [x+"_d"+str(y) for x in code for y in range(d)]
twitter_d = [x+"_d"+str(y) for x in twitter for y in range(d)]
reddit_d = [x+"_d"+str(y) for x in reddit for y in range(d)]
fb_d = [x+"_d"+str(y) for x in fb for y in range(d)]
market_d = [x+"_d"+str(y) for x in market for y in range(d)]
timestamp = ['timestamp'+'_d'+str(y)  for y in range(d)]

In [21]:
#Omit timestamp for now
edited_cols = np.array(core + code_d + twitter_d + reddit_d + fb_d + market_d)
#MUST MAKEE SURE THE ELEMENTS ARE IN THE SAME ORDER AS IN COL_DICT ABOVE!!!!
social_features = np.array(code_d + twitter_d + reddit_d + fb_d + market_d)
print(len(edited_cols))
unique_cols = np.unique(edited_cols)
print(len(unique_cols))

1065
1065


## Make sure all the importmnat columns are preserved

In [22]:
orig_cols = x_final.columns
for i in orig_cols:
    if i not in unique_cols:
        print(i)

timestamp_d0
timestamp_d1
timestamp_d2
timestamp_d3
timestamp_d4
timestamp_d5
timestamp_d6
timestamp_d7
timestamp_d8
timestamp_d9
timestamp_d10
timestamp_d11
timestamp_d12
timestamp_d13
timestamp_d14
timestamp_d15
timestamp_d16
timestamp_d17
timestamp_d18
timestamp_d19
timestamp_d20
timestamp_d21
timestamp_d22
timestamp_d23
timestamp_d24
timestamp_d25
timestamp_d26
timestamp_d27
timestamp_d28
timestamp_d29
timestamp_d30
timestamp_d31
date_d0
date_utc_d0
coin_name_d0
coin_id_d0
date_d1
date_utc_d1
coin_name_d1
coin_id_d1
date_d2
date_utc_d2
coin_name_d2
coin_id_d2
date_d3
date_utc_d3
coin_name_d3
coin_id_d3
date_d4
date_utc_d4
coin_name_d4
coin_id_d4
date_d5
date_utc_d5
coin_name_d5
coin_id_d5
date_d6
date_utc_d6
coin_name_d6
coin_id_d6
date_d7
date_utc_d7
coin_name_d7
coin_id_d7
date_d8
date_utc_d8
coin_name_d8
coin_id_d8
date_d9
date_utc_d9
coin_name_d9
coin_id_d9
date_d10
date_utc_d10
coin_name_d10
coin_id_d10
date_d11
date_utc_d11
coin_name_d11
coin_id_d11
date_d12
date_utc_d12
coin

In [23]:
x_final = x_final[edited_cols]
x_final = x_final.reset_index(drop=True)

In [30]:
x_final.to_pickle("social_data_training_d.pkl")

### Also save the column names just to be sure


In [31]:
f = open("col_dict_d.pkl","wb")
pickle.dump(col_dict,f)
f.close()

g = open("social_features_d.pkl","wb")
pickle.dump(social_features,g)
g.close()

In [32]:
x_final.head()

,channelLink,channelTitle,currency,exchange,timestamp,signalTime,pumptime_edited,coin_name,coin_id,code_repo_stars_d0,code_repo_stars_d1,code_repo_stars_d2,code_repo_stars_d3,code_repo_stars_d4,code_repo_stars_d5,code_repo_stars_d6,code_repo_stars_d7,code_repo_stars_d8,code_repo_stars_d9,code_repo_stars_d10,code_repo_stars_d11,code_repo_stars_d12,code_repo_stars_d13,code_repo_stars_d14,code_repo_stars_d15,code_repo_stars_d16,code_repo_stars_d17,code_repo_stars_d18,code_repo_stars_d19,code_repo_stars_d20,code_repo_stars_d21,code_repo_stars_d22,code_repo_stars_d23,code_repo_stars_d24,code_repo_stars_d25,code_repo_stars_d26,code_repo_stars_d27,code_repo_stars_d28,code_repo_stars_d29,code_repo_stars_d30,code_repo_stars_d31,code_repo_forks_d0,code_repo_forks_d1,code_repo_forks_d2,code_repo_forks_d3,code_repo_forks_d4,code_repo_forks_d5,code_repo_forks_d6,code_repo_forks_d7,code_repo_forks_d8,code_repo_forks_d9,code_repo_forks_d10,code_repo_forks_d11,code_repo_forks_d12,code_repo_forks_d13,code_repo_forks_d14,code_repo_forks_d15,code_repo_forks_d16,code_repo_forks_d17,code_repo_forks_d18,code_repo_forks_d19,code_repo_forks_d20,code_repo_forks_d21,code_repo_forks_d22,code_repo_forks_d23,code_repo_forks_d24,code_repo_forks_d25,code_repo_forks_d26,code_repo_forks_d27,code_repo_forks_d28,code_repo_forks_d29,code_repo_forks_d30,code_repo_forks_d31,code_repo_subscribers_d0,code_repo_subscribers_d1,code_repo_subscribers_d2,code_repo_subscribers_d3,code_repo_subscribers_d4,code_repo_subscribers_d5,code_repo_subscribers_d6,code_repo_subscribers_d7,code_repo_subscribers_d8,code_repo_subscribers_d9,code_repo_subscribers_d10,code_repo_subscribers_d11,code_repo_subscribers_d12,code_repo_subscribers_d13,code_repo_subscribers_d14,code_repo_subscribers_d15,code_repo_subscribers_d16,code_repo_subscribers_d17,code_repo_subscribers_d18,code_repo_subscribers_d19,code_repo_subscribers_d20,code_repo_subscribers_d21,code_repo_subscribers_d22,code_repo_subscribers_d23,code_repo_subscribers_d24,code_repo_subscribers_d25,code_repo_subscribers_d26,code_repo_subscribers_d27,code_repo_subscribers_d28,code_repo_subscribers_d29,code_repo_subscribers_d30,code_repo_subscribers_d31,code_repo_contributors_d0,code_repo_contributors_d1,code_repo_contributors_d2,code_repo_contributors_d3,code_repo_contributors_d4,code_repo_contributors_d5,code_repo_contributors_d6,code_repo_contributors_d7,code_repo_contributors_d8,code_repo_contributors_d9,code_repo_contributors_d10,code_repo_contributors_d11,code_repo_contributors_d12,code_repo_contributors_d13,code_repo_contributors_d14,code_repo_contributors_d15,code_repo_contributors_d16,code_repo_contributors_d17,code_repo_contributors_d18,code_repo_contributors_d19,code_repo_contributors_d20,code_repo_contributors_d21,code_repo_contributors_d22,code_repo_contributors_d23,code_repo_contributors_d24,code_repo_contributors_d25,code_repo_contributors_d26,code_repo_contributors_d27,code_repo_contributors_d28,code_repo_contributors_d29,code_repo_contributors_d30,code_repo_contributors_d31,code_repo_open_pull_issues_d0,code_repo_open_pull_issues_d1,code_repo_open_pull_issues_d2,code_repo_open_pull_issues_d3,code_repo_open_pull_issues_d4,code_repo_open_pull_issues_d5,code_repo_open_pull_issues_d6,code_repo_open_pull_issues_d7,code_repo_open_pull_issues_d8,code_repo_open_pull_issues_d9,code_repo_open_pull_issues_d10,code_repo_open_pull_issues_d11,code_repo_open_pull_issues_d12,code_repo_open_pull_issues_d13,code_repo_open_pull_issues_d14,code_repo_open_pull_issues_d15,code_repo_open_pull_issues_d16,code_repo_open_pull_issues_d17,code_repo_open_pull_issues_d18,code_repo_open_pull_issues_d19,code_repo_open_pull_issues_d20,code_repo_open_pull_issues_d21,code_repo_open_pull_issues_d22,code_repo_open_pull_issues_d23,code_repo_open_pull_issues_d24,code_repo_open_pull_issues_d25,code_repo_open_pull_issues_d26,code_repo_open_pull_issues_d27,code_repo_open_pull_issues_d28,code_repo_open_pull_issues_d29,code_repo_open_pull_issues_d30,code_repo_open

In [29]:
social_features

array(['code_repo_stars_d0', 'code_repo_stars_d1', 'code_repo_stars_d2',
       ..., 'total_page_views_d29', 'total_page_views_d30',
       'total_page_views_d31'], dtype='<U32')